In [1]:
import pandas as pd
import itertools
df = pd.read_csv('kobe.csv')

In [4]:
df1 = df.groupby('opponent').get_group('HOU')
df2 = df1.groupby(['shot_type','shot_made_flag'])
two_rate = len(df2.groups['2PT Field Goal',1])/(len(df2.groups['2PT Field Goal',0]) + len(df2.groups['2PT Field Goal',1]))
three_rate = len(df2.groups['3PT Field Goal',1])/(len(df2.groups['3PT Field Goal',0]) + len(df2.groups['3PT Field Goal',1]))
print('Kobe 對戰火箭隊(HOU)的平均兩分球與三分球命中率：')
print('2PT Field Goal: %.3f' % two_rate)
print('3PT Field Goal: %.3f' % three_rate)

Kobe 對戰火箭隊(HOU)的平均兩分球與三分球命中率：
2PT Field Goal: 0.388
3PT Field Goal: 0.266


In [15]:
#將二分球三分球轉為數字，過濾有命中的，再計算與各隊的平均得分
df1 = df.replace({'2PT Field Goal': 2,'3PT Field Goal': 3})
df2 = df1.query('shot_made_flag == 1')
df3 = df2.groupby('opponent')
list1 = []
for opponent,df4 in df3:
    df5 = df4.groupby('game_id')
    point = df4.sum()['shot_type']
    list2 = [point/len(df5),opponent]
    list1.append(list2)
print('Kobe 對戰過的球隊中，使得 Kobe 平均得分最低的前五支球隊與 Kobe 的平均得分：')
list1.sort()
for i in range(5):
    print(i+1,list1[i][1],'%.3f' % list1[i][0])

Kobe 對戰過的球隊中，使得 Kobe 平均得分最低的前五支球隊與 Kobe 的平均得分：
1 VAN 11.167
2 BKN 12.667
3 NJN 14.179
4 MIL 14.613
5 CLE 14.727


In [8]:
#根據條件過濾，sort得分印出前三高
df1 = df.query('minutes_remaining < 3 & period == 4 & shot_made_flag == 1 & playoffs == 1')
df2 = df1.replace({'2PT Field Goal': 2,'3PT Field Goal': 3})
df3 = df2.groupby('game_id').sum()
df4 = df3.sort_values(by = 'shot_type',ascending = False)
print('Kobe 在季後賽最後 3 分鐘內得分最高的前五場球賽與其得分：')
for i in range(5):
    print('id =',df4.index[i],',','point =',df4['shot_type'].iloc[i])

Kobe 在季後賽最後 3 分鐘內得分最高的前五場球賽與其得分：
id = 49900074 , point = 9
id = 40100234 , point = 8
id = 49900024 , point = 7
id = 40700402 , point = 7
id = 49600024 , point = 6


In [17]:
#根據條件過濾，按照日期排序球季
df1 = df.query('minutes_remaining < 1 & playoffs == 1')
df2 = df1.sort_values(by = 'game_date')
df3 = df2.groupby('season',sort = False)
print('Kobe 各球季季後賽中，比賽最後 1 分鐘內的 Jump Shot 命中率（按球季先後排序）：')
for game_date,df4 in df3:
    df5 = df4.query('shot_made_flag == 1') 
    print(game_date,':','%.3f' % (df5.shape[0]/df4.shape[0]))

Kobe 各球季季後賽中，比賽最後 1 分鐘內的 Jump Shot 命中率（按球季先後排序）：
1996-97 : 0.214
1997-98 : 0.286
1998-99 : 0.312
1999-00 : 0.333
Jan-00 : 0.324
Feb-01 : 0.241
Mar-02 : 0.257
Apr-03 : 0.197
Jun-05 : 0.400
Jul-06 : 0.400
Aug-07 : 0.326
Sep-08 : 0.344
Oct-09 : 0.328
Nov-10 : 0.235
Dec-11 : 0.267


In [11]:
df1 = df.copy()
df1['game_date'] = pd.to_datetime(df1['game_date'],errors='coerce')
df2 = df1.query('shot_made_flag == 1')
dict1 = df2.groupby('game_date').groups
dict2 = df1.groupby('game_date').groups
game_date_list = list(dict2.keys())
hit_rate = []
for game_date in dict2:
    hit_rate1 = []
    #整場都沒命中
    if game_date not in dict1:
        hit_rate1 = [game_date,0]
    #命中除以沒命中
    else:
        hit_rate1 = [game_date,len(dict1[game_date])/len(dict2[game_date])]
    hit_rate.append(hit_rate1)
continue_dict = {}
continue_time = 0
#有連續得分率0.33以上就繼續加，沒有則連續次序歸零
for i in range(1,len(hit_rate)):
    if hit_rate[i][1] > 0.33:
        continue_time += 1
    elif continue_time > 0:
        continue_dict[hit_rate[i-1][0]] = continue_time
        continue_time = 0
#印出連續最長次數前三
print("Kobe「得分命中率 33%以上的最長連續場數」前 3 名，其場數以及起訖日期：")
for i in range(3):
    max_continue = max(continue_dict.values())
    for date in continue_dict:
        if continue_dict[date] == max_continue:
            last_date_index = game_date_list.index(date)
            start_date = game_date_list[last_date_index-max_continue+1]
            print(max_continue,start_date.strftime('%Y-%m-%d'),"~",date.strftime('%Y-%m-%d'))
            del continue_dict[date]
            break

Kobe「得分命中率 33%以上的最長連續場數」前 3 名，其場數以及起訖日期：
19 2000-02-01 ~ 2000-03-12
19 2001-06-15 ~ 2001-12-07
16 2002-01-14 ~ 2002-02-17


In [18]:
df['period'].replace({1:'first',2:'first',3:'second',4:'second'},inplace = True)
df1 = df.query('period != 5')
df2 = df1.replace({'2PT Field Goal': 2,'3PT Field Goal': 3})
df3 = df2.groupby('game_date')
list1 = []
for game_date,df4 in df3:
    df5 = df4.query('shot_made_flag == 1')
    df6 = df5.groupby('period').sum()
    if len(df6['shot_type']) > 1:
        if df6['shot_type']['first'] > df6['shot_type']['second']:
            difference = df6['shot_type']['first'] - df6['shot_type']['second']
            score = df6['shot_type']['first'] + df6['shot_type']['second']
            list2 = ['%.3f' % (df5.shape[0]/df4.shape[0]),game_date,df4['opponent'].iloc[0],difference,score]
            list1.append(list2)
print('Kobe「上半場得分多於下半場」且命中率最低的前 3 名場次，其日期、對手、上 下半場得分差、該場得分，按命中率由上而下排序')
list1.sort()
for i in range(3):
    for j in range(5):
        print(list1[i][j],end = ' ')
    print()

Kobe「上半場得分多於下半場」且命中率最低的前 3 名場次，其日期、對手、上 下半場得分差、該場得分，按命中率由上而下排序
0.100 2012/3/18 UTA 1 5 
0.130 2004/4/6 POR 2 6 
0.130 2012/11/23 MEM 2 8 
